In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D

Using TensorFlow backend.


In [ ]:
# We'll define a classifier that takes our images and convolves then, then feeds the obtained feature maps
# to a neural network that then classifies the images into a binary class.

# First Layer
# - Convolve the image from 64x63 in 3 channels (RGB) with the relu layer and create random feature filters
# Second Layer
# - Pool the data from the previous layer and only leave the max value of a feature set, this prevents noise.
# Third Layer
# - Convolve the resulting data again with random feature filters, this seems to have a positive effect on accuracy.
# Fourth Layer
# - Pool the resulting data again and only leave the maximum value of a feature set, again preventing noise.
# Fifth Layer
# - Flatten the data into a single dimension array from the grid we've been working in
# Sixt layer
# - Now we use an input layer with our 64x64=128 data points as the first layer of our neural network, and apply
#   and apply the relu activation function.
# Final Layer
# - take the output of the neural network and give a probability of the network having recognised the image.
classifier = Sequential([
    Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dense(units=1, activation='sigmoid')
])
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# first we should generate more training data to have our network recognise images even if they are rescaled, sheared,
# zoomed or flipped.

# Initialise the image generators
train_data_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_data_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
# generate the training and test set from existing images and generated ones.
training_set = train_data_generator.flow_from_directory(
    'datasets/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)
test_set = test_data_generator.flow_from_directory(
    'datasets/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
# now we fit the images to the classifier
classifier.fit_generator(
    training_set,
    steps_per_epoch=1,
    nb_epoch=5,
    validation_data=test_set,
    validation_steps=2000
)